In [1]:
import numpy as np

In [2]:
d = np.loadtxt(r"C:\Users\admin\Desktop\Prediction-Simulation-Data\SIMU_RESULT_2000.csv", delimiter=',')

In [3]:
alpha = (d[:, 1:] > 0).astype(int)

In [4]:
deg = np.sum(alpha, axis=1)

In [5]:
# init_d = d[:, 1:]
# w = np.zeros((init_d.shape[0], init_d.shape[1]))
# for j in range(init_d.shape[0]):
#     for l in range(init_d.shape[1]):
#         w[j, l] = np.inner(alpha[:, j], alpha[:, l] / deg) / deg[l]

In [6]:
import pickle

In [7]:
# with open(r"C:\Users\admin\Desktop\Prediction-Simulation-Data\WEIGHT_BGRAPH", "wb") as outfile:
#     pickle.dump(w, outfile)
# with open(r"C:\Users\admin\Desktop\Prediction-Simulation-Data\INIT_D", "wb") as outfile:
#     pickle.dump(init_d, outfile)

In [8]:
with open(r"C:\Users\admin\Desktop\Prediction-Simulation-Data\WEIGHT_BGRAPH", "rb") as outfile:
    w = pickle.load(outfile)
with open(r"C:\Users\admin\Desktop\Prediction-Simulation-Data\INIT_D", "rb") as outfile:
    init_d = pickle.load(outfile)

In [9]:
new_d = w @ init_d

In [10]:
import csv
OD_NUM = 2000
OD_filepath = r"C:\Users\admin\Desktop\Prediction-Simulation-Data\OD_2000.csv"
OD_pairs = np.zeros((OD_NUM, 6))
with open(OD_filepath, mode='r') as file:
    data = csv.reader(file)
    head = next(data)
    id_col = head.index("id")
    x1_col = head.index("originX")
    y1_col = head.index("originY")
    x2_col = head.index("destX")
    y2_col = head.index("destY")
    lam_col = head.index("lambda")
    j = 0
    for row in data:
        x1 = int(row[x1_col])
        x2 = int(row[x2_col])
        y1 = int(row[y1_col])
        y2 = int(row[y2_col])
        lam = float(row[lam_col])
        OD_pairs[j, :] = np.array([[j, x1, y1, x2, y2, lam]])
        j += 1
        if j >= OD_NUM:
            break

In [11]:
MAX_OD_NUM = 100
for n in range(MAX_OD_NUM):
    new_OD_pairs = OD_pairs[new_d[n, :] > 0, :]
    np.savetxt(r"C:\Users\admin\Desktop\误差测试\OD_Cluster_" + str(n) + ".csv", new_OD_pairs, fmt=('%d', '%d', '%d', '%d', '%d', '%.2f'), delimiter=',', header="id, originX, originY, destX, destY, lambda", comments='')

In [12]:
index = -np.ones(MAX_OD_NUM)
for n in range(MAX_OD_NUM):
    index[n] = np.sum(new_d[n, :n] > 0)

In [13]:
predict_result = np.zeros((MAX_OD_NUM, 3))
for n in range(MAX_OD_NUM):
    with open(r"C:\Users\admin\Desktop\误差测试\OD_Cluster_Result_" + str(n) + ".csv", mode='r') as file:
        data = csv.reader(file)
        head = next(data)
        pw_col = head.index("Pw")
        lw_col = head.index("Lw")
        ew_col = head.index("ew")
        j = 0
        for row in data:
            if j < index[n]:
                j += 1
                continue
            pw = float(row[pw_col])
            lw = float(row[lw_col])
            ew = float(row[ew_col])
            predict_result[n, 0] = pw
            predict_result[n, 1] = lw
            predict_result[n, 2] = ew
            break

In [14]:
OD_filepath = r"C:\Users\admin\Desktop\Prediction-Simulation-Data\RESULT_2000.csv"
global_predict_result = np.zeros((MAX_OD_NUM, 3))
with open(OD_filepath, mode='r') as file:
    data = csv.reader(file)
    head = next(data)
    pw_col = head.index("Pw")
    lw_col = head.index("Lw")
    ew_col = head.index("ew")
    j = 0
    for row in data:
        pw = float(row[pw_col])
        lw = float(row[lw_col])
        ew = float(row[ew_col])
        global_predict_result[j, 0] = pw
        global_predict_result[j, 1] = lw
        global_predict_result[j, 2] = ew
        j += 1
        if j >= MAX_OD_NUM:
            break

In [15]:
rel_error = np.abs(global_predict_result - predict_result) / global_predict_result

In [16]:
np.mean(rel_error, axis=0)

array([0.00122321, 0.00096853, 0.00321566])

In [17]:
np.max(rel_error, axis=0)

array([0.00975918, 0.00566872, 0.05738011])

In [18]:
np.sum((new_d > 10)) / new_d.shape[0]

0.85

In [19]:
np.sum((new_d > 1e-1)) / new_d.shape[0]

138.983